In [3]:
import numpy as np
import cv2
import pandas as pd
import os
from tqdm import tqdm
trim_csv =r"C:\Users\jingy\Parkinson\Codes\data_trim.csv"
df= pd.read_csv(trim_csv)

In [4]:
df

,Video_Path,Time,ON or OFF,R1 or R2,Halfway or Wholeway,FW_Start,FW_End,BW_Start,BW_End,Video Condition
0,"""C:\Users\jingy\Parkinson\NIHFX695VBHFM\Baseli...",Baseline,OFF,R1,Whole,12.00,18.44,20.68,25.08,NaN
1,"""C:\Users\jingy\Parkinson\NIHFX695VBHFM\Baseli...",Baseline,OFF,R2,Whole,168.64,180.00,189.52,199.08,NaN
2,"""C:\Users\jingy\Parkinson\NIHFX695VBHFM\Month ...",M12,OFF,R1,Half,NaN,NaN,25.92,31.60,NaN
3,"""C:\Users\jingy\Parkinson\NIHFX695VBHFM\Month ...",M12,OFF,R2,Whole,150.52,161.48,174.32,184.72,NaN
4,"""C:\Users\jingy\Parkinson\NIHFX695VBHFM\Month ...",M12,ON,R1,Half,15.96,21.20,23.56,29.04,NaN
...,...,...,...,...,...,...,...,...,...,...
464,"""C:\Users\jingy\Parkinson\NIHYW345FJFDF\Baseli...",Baseline,ON,R2,Half,90.00,96.00,NaN,NaN,NaN
465,"""C:\Users\jingy\Parkinson\NIHZB051YHETP\Baseli...",Baseline,OFF,R1,Whole,148.00,157.00,172.00,178.50,NaN
466,"""C:\Users\jingy\Parkinson\NIHZB051YHETP\Baseli...",Baseline,OFF,R2,Half,183.00,192.00,NaN,NaN,NaN
467,"""C:\Users\jingy\Parkinson\NIHZB051YHETP\Baseli...",Baseline,ON,R1,Whole,76.00,84.00,99.00,105.50,NaN


In [5]:
def check_trim_csv(df):
    for index, row in df.iterrows():
        # make sure column name is corrects(without extra space)
        vid_path = row.get("Video_Path")
        FW_Start, FW_End = row.get("FW_Start"), row.get("FW_End")
        BW_Start, BW_End = row.get("BW_Start"), row.get("BW_End")
        vid_path = vid_path.replace('"','')
        #check if video exist
        try:
            os.path.exists(vid_path)
        except:
            print("File not exists:", vid_path)
        #check if the label is correct
        if not pd.isnull(FW_Start) and not pd.isnull(FW_End):
            try:
                assert FW_Start< FW_End
                assert FW_End-FW_Start < 20
            except:
                print("Incorrect Time", index)
                
        if not pd.isnull(BW_Start) and not pd.isnull(BW_End):
            try:
                assert BW_Start< BW_End
                assert BW_End-BW_Start < 20
            except:
                print("Incorrect Time", index)
check_trim_csv(df)

In [ ]:
def trim_videos(df):
    for index, row in tqdm(df.iterrows()):
        # make sure column name is corrects(without extra space)
        vid_path = row.get("Video_Path")
        vid_path = vid_path.replace('"','')
        FW_Start, FW_End = row.get("FW_Start"), row.get("FW_End")
        BW_Start, BW_End = row.get("BW_Start"), row.get("BW_End")
        # define some output directories and video_name
        data_dir = os.path.join(os.getcwd(),"dataset")
        subject_dir = os.path.join(data_dir,vid_path.split("\\")[4])
        # make folder
        try:
            os.makedirs(subject_dir)
        except:
            pass
        
        if not pd.isnull(FW_Start) and not pd.isnull(FW_End):
            output_vid = os.path.join(subject_dir, row["Time"]+"_"+"FW_"+row["R1 or R2"]+"_"+str(FW_Start)+"_.avi")
            if not os.path.exists(output_vid):
                trim_vid(vid_path,output_vid,FW_Start,FW_End)
        
        if not pd.isnull(BW_Start) and not pd.isnull(BW_End):
            output_vid = os.path.join(subject_dir, row["Time"]+"_"+"BW_"+row["R1 or R2"]+"_"+str(BW_Start)+"_.avi")
            if not os.path.exists(output_vid):
                trim_vid(vid_path,output_vid,BW_Start,BW_End)



trim_videos(df)


In [6]:
def check_vid(vid_path,start,end):
    # define some video property 
    cap = cv2.VideoCapture(vid_path)
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    # frame_width = int(cap.get(3))
    # frame_height = int(cap.get(4))
    # frame_num = int(cap.get(7))

    # calculate the frame to start and end 
    start_frame = int(start * fps)
    end_frame = int(end * fps)

    # define out put video

    # out = cv2.VideoWriter(output_vid,cv2.VideoWriter_fourcc(*'MJPG'), 30,
    # (frame_width,frame_height))

    # set the video to start frame
    cap.set(1,start_frame)
    ret, frame = cap.read()

    if ret != True:
        print("unreadable: "+vid_path)
    if int(fps) == 0:
        print("fps equals 0: "+vid_path)
    

    cap.release()


In [7]:
def check_videos(df):
    # check input videos
    for index, row in tqdm(df.iterrows()):
        # make sure column name is corrects(without extra space)
        vid_path = row.get("Video_Path")
        vid_path = vid_path.replace('"','')
        FW_Start, FW_End = row.get("FW_Start"), row.get("FW_End")
        BW_Start, BW_End = row.get("BW_Start"), row.get("BW_End")
        # define some output directories and video_name
        data_dir = os.path.join(os.getcwd(),"dataset")
        subject_dir = os.path.join(data_dir,vid_path.split("\\")[4])
        # make folder

        
        if not pd.isnull(FW_Start) and not pd.isnull(FW_End):
            check_vid(vid_path,FW_Start,FW_End)
        
        if not pd.isnull(BW_Start) and not pd.isnull(BW_End):

            check_vid(vid_path,BW_Start,BW_End)



check_videos(df)


87it [00:31,  3.05it/s]

unreadable: C:\Users\jingy\Parkinson\NIHDB604WURZN\Month 12\Gait.MTS


88it [00:36,  1.52s/it]

unreadable: C:\Users\jingy\Parkinson\NIHDB604WURZN\Month 12\Gait.MTS


89it [00:37,  1.40s/it]

unreadable: C:\Users\jingy\Parkinson\NIHDB604WURZN\Month 12\Gait.MTS


469it [02:59,  2.61it/s]


In [9]:
def check_ouput_vid(df):
    for index, row in tqdm(df.iterrows()):
        # make sure column name is corrects(without extra space)
        vid_path = row.get("Video_Path")
        vid_path = vid_path.replace('"','')
        FW_Start, FW_End = row.get("FW_Start"), row.get("FW_End")
        BW_Start, BW_End = row.get("BW_Start"), row.get("BW_End")
        # define some output directories and video_name
        data_dir = os.path.join(os.getcwd(),"dataset")
        subject_dir = os.path.join(data_dir,vid_path.split("\\")[4])
        # make folder
        
        if not pd.isnull(FW_Start) and not pd.isnull(FW_End):
            output_vid = os.path.join(subject_dir, row["Time"]+"_"+"FW_"+row["R1 or R2"]+"_"+str(FW_Start)+"_.avi")
            cap = cv2.VideoCapture(output_vid)
            fps = int(cap.get(cv2.CAP_PROP_FPS))
            ret, frame = cap.read()

            if ret != True:
                print("unreadable: "+output_vid)
            if int(fps) == 0:
                print("fps equals 0: "+output_vid)
        
        if not pd.isnull(BW_Start) and not pd.isnull(BW_End):
            output_vid = os.path.join(subject_dir, row["Time"]+"_"+"BW_"+row["R1 or R2"]+"_"+str(BW_Start)+"_.avi")
            cap = cv2.VideoCapture(output_vid)
            fps = int(cap.get(cv2.CAP_PROP_FPS))
            ret, frame = cap.read()

            if ret != True:
                print("unreadable: "+output_vid)
            if int(fps) == 0:
                print("fps equals 0: "+output_vid)
check_ouput_vid(df)

100it [00:01, 63.15it/s]

unreadable: c:\Users\jingy\Parkinson\Codes\dataset\NIHDB604WURZN\M12_FW_R1_55.0_.avi
unreadable: c:\Users\jingy\Parkinson\Codes\dataset\NIHDB604WURZN\M12_BW_R1_80.0_.avi
unreadable: c:\Users\jingy\Parkinson\Codes\dataset\NIHDB604WURZN\M12_FW_R2_90.0_.avi


469it [00:08, 56.50it/s]


In [8]:
def check_ouput_vid(df):
    for index, row in tqdm(df.iterrows()):
        # make sure column name is corrects(without extra space)
        vid_path = row.get("Video_Path")
        vid_path = vid_path.replace('"','')
        FW_Start, FW_End = row.get("FW_Start"), row.get("FW_End")
        BW_Start, BW_End = row.get("BW_Start"), row.get("BW_End")
        # define some output directories and video_name
        data_dir = os.path.join(os.getcwd(),"dataset")
        subject_dir = os.path.join(data_dir,vid_path.split("\\")[4])
        # make folder
        
        if not pd.isnull(FW_Start) and not pd.isnull(FW_End):
            output_vid = os.path.join(subject_dir, row["Time"]+"_"+"FW_"+row["R1 or R2"]+"_"+str(FW_Start)+"_.avi")
            cap = cv2.VideoCapture(vid_path)
            fps = int(cap.get(cv2.CAP_PROP_FPS))
            if fps!=30:
                print("FPS:", fps, vid_path)

            ret, frame = cap.read()
            if ret != True:
                print("unreadable: "+vid_path)
            if int(fps) == 0:
                print("fps equals 0: "+vid_path)
        
        if not pd.isnull(BW_Start) and not pd.isnull(BW_End):
            output_vid = os.path.join(subject_dir, row["Time"]+"_"+"BW_"+row["R1 or R2"]+"_"+str(BW_Start)+"_.avi")
            cap = cv2.VideoCapture(vid_path)
            fps = int(cap.get(cv2.CAP_PROP_FPS))
            ret, frame = cap.read()
            if fps!=30:
                print("FPS:", fps, vid_path)
            if ret != True:
                print("unreadable: "+vid_path)
            if int(fps) == 0:
                print("fps equals 0: "+vid_path)
check_ouput_vid(df)

1it [00:00,  4.75it/s]

FPS: 29 C:\Users\jingy\Parkinson\NIHFX695VBHFM\Baseline (No ON assessment - forgot meds)\OFF Gait.MP4
FPS: 29 C:\Users\jingy\Parkinson\NIHFX695VBHFM\Baseline (No ON assessment - forgot meds)\OFF Gait.MP4
FPS:

2it [00:00,  4.74it/s]

 29 C:\Users\jingy\Parkinson\NIHFX695VBHFM\Baseline (No ON assessment - forgot meds)\OFF Gait.MP4
FPS: 29 C:\Users\jingy\Parkinson\NIHFX695VBHFM\Baseline (No ON assessment - forgot meds)\OFF Gait.MP4


3it [00:00,  6.15it/s]

...

FPS: 29 C:\Users\jingy\Parkinson\NIHYW345FJFDF\Baseline\NIHYW345FJFDF ON pt3.MP4
FPS: 29 C:\Users\jingy\Parkinson\NIHYW345FJFDF\Baseline\NIHYW345FJFDF ON pt3.MP4
FPS: 29 C:\Users\jingy\Parkinson\NIHZB051YHETP\Baseline Visit\NIHZB051YHETP_OFF_Part_2.MP4


467it [01:22,  6.35it/s]

FPS: 29 C:\Users\jingy\Parkinson\NIHZB051YHETP\Baseline Visit\NIHZB051YHETP_OFF_Part_2.MP4
FPS: 29 C:\Users\jingy\Parkinson\NIHZB051YHETP\Baseline Visit\NIHZB051YHETP_OFF_Part_2.MP4
FPS: 29 C:\Users\jingy\Parkinson\NIHZB051YHETP\Baseline Visit\NIHZB051YHETP_ON_Part_2.MP4


469it [01:23,  5.65it/s]

FPS: 29 C:\Users\jingy\Parkinson\NIHZB051YHETP\Baseline Visit\NIHZB051YHETP_ON_Part_2.MP4
FPS: 29 C:\Users\jingy\Parkinson\NIHZB051YHETP\Baseline Visit\NIHZB051YHETP_ON_Part_2.MP4
